## Understanding Simple Testing Techniques

#### Unit Test

In [32]:
from typing import List # for type hinting (req. in Python versions < 3.9)

class CakeFactory:
    def __init__(self, cake_type: str, size: str):
        self.cake_type = cake_type
        self.size = size
        self.toppings = []
    
        # price based on cake type and size:
        self.price = 10 if self.cake_type == "chocolate" else 8
        self.price += 2 if self.size == "medium" else 4 if self.size == "large" else 0

    def add_topping(self, topping: str):
        self.toppings.append(topping)
        self.price += 1 # charging 1 unit for each topping

    def check_ingredients(self) -> List[str]:
        ingredients = ['flour', 'sugar', 'eggs']
        ingredients.append('cocoa') if self.cake_type == "chocolate" else ingredients.append('vanilla extract')
        ingredients += self.toppings
        return ingredients

    def check_price(self) -> float:
        return self.price


# example of creating a cake and adding toppings:
cake = CakeFactory("chocolate", "medium")
cake.add_topping("sprinkles")
cake.add_topping("cherries")
cake_ingredients = cake.check_ingredients()
cake_price = cake.check_price()

cake_ingredients, cake_price # prints results for both (in notebook only)

(['flour', 'sugar', 'eggs', 'cocoa', 'sprinkles', 'cherries'], 14)

In [ ]:
import unittest # using in-built module for unit-testing

# creating a 'child class' of 'TestCase' class (in order to run our tests):
class TestCakeFactory(unittest.TestCase):
    def test_create_cake(self):
        cake = CakeFactory("vanilla", "small")

        self.assertEqual(cake.cake_type, "vanilla")
        self.assertEqual(cake.size, "small")
        self.assertEqual(cake.price, 8) # Vanilla cake, small size

    def test_add_topping(self):
        cake = CakeFactory("chocolate", "large")
        cake.add_topping("sprinkles")

        self.assertIn("sprinkles", cake.toppings)

    def test_check_ingredients(self):
        cake = CakeFactory("chocolate", "medium")
        cake.add_topping("cherries")
        ingredients = cake.check_ingredients()

        self.assertIn("cocoa", ingredients)
        self.assertIn("cherries", ingredients)
        self.assertNotIn("vanilla extract", ingredients)

    def test_check_price(self):
        cake = CakeFactory("vanilla", "large")
        cake.add_topping("sprinkles")
        cake.add_topping("cherries")
        price = cake.check_price()
     
        self.assertEqual(price, 14) # Vanilla cake, large size + 2 toppings


# running the unit-tests:
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestCakeFactory)) # temporary object creation

....
----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

**_related material:_** \
1\. [understanding class inheritance in python](https://www.geeksforgeeks.org/inheritance-in-python/) \
2\. [important concepts for understanding unit test module](unit_test_module_concepts.md)

#### Py Test

In [ ]:
# installing pytest library (with shell command)
! pip install pytest

In [35]:
# changing the current working directory (with line magic command) 
%cd ../__scripts/testing_techniques/

/mnt/c/Users/SHAHZAIB AHMED/OneDrive/Desktop/shared_work/Google_IT_Automation_with_Python/c2_python_operating_system/__scripts/testing_techniques


In [36]:
%%file using_pytest.py
import pytest # external library for testing
from typing import List # for type hinting

class Fruit:
    def __init__(self, name: str):
        self.name = name
        self.cubed = False
    
    def cube(self):
        self.cubed = True


class FruitSalad:
    def __init__(self, fruit_bowl: List[Fruit]):
        self.fruits = fruit_bowl
        self._cube_fruit()
     
    def _cube_fruit(self):
        for fruit in self.fruits:
            fruit.cube()


# Arrange (setup) and Act (execution) steps:
@pytest.fixture
def fruit_bowl(): # arrange
    return [Fruit('apple'), Fruit('banana')]

def test_fruit_salad(fruit_bowl):
    fruit_salad = FruitSalad(fruit_bowl) # act
    assert all(fruit.cubed for fruit in fruit_salad.fruits) # assert


# note: %%file -> cell magic command for saving the cell content to a file

# side-notes (for pytest):
# 1) a 'fixture' is a function that establishes a fixed baseline for tests
#    by initializing data & preparing the testing environment
# 2) any function name starting with 'test_' will be considered as a test function

Writing using_pytest.py


In [37]:
# running the test script with pytest
! pytest -v using_pytest.py # prints the results of each test (verbose mode)

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0 -- /bin/python3
cachedir: .pytest_cache
rootdir: /mnt/c/Users/SHAHZAIB AHMED/OneDrive/Desktop/shared_work/Google_IT_Automation_with_Python/c2_python_operating_system/__scripts/testing_techniques
collected 1 item                                                               

using_pytest.py::test_fruit_salad PASSED                                 [100%]

============================== 1 passed in 0.13s ===============================


In [38]:
# changing current working directory back to previous one
%cd -

/mnt/c/Users/SHAHZAIB AHMED/OneDrive/Desktop/shared_work/Google_IT_Automation_with_Python/c2_python_operating_system/__important_notes


**_additional material:_** \
1\. [shell (!) vs magic (%) commands in jupyter notebook](https://stackoverflow.com/questions/45784499/what-is-the-difference-between-and-in-jupyter-notebooks) \
2\. [line (%) vs cell (%%) magic commands in jupyter notebook](https://www.stata.com/python/pystata18/notebook/Magic%20Commands0.html#:~:text=line,(%25%25).)


#### Understanding Test suites with setUp() and teaarDown()

In [26]:
import unittest, os, shutil

# Function to test
def simple_addition(a, b): return a + b

# Paths for file operations
ORIGINAL_FILE_PATH = "/tmp/original_test_file.txt"
COPIED_FILE_PATH = "../__assets/copied_test_file.txt"

# This method will be run once before any tests or test classes
def setUpModule():
	global COUNTER
	COUNTER = 0
    
	# Create a file in /tmp
	with open(ORIGINAL_FILE_PATH, 'w') as file:
		file.write("Test Results:\n")

# This method will be run once after all tests and test classes
def tearDownModule():
	# Copy the file to another directory
	shutil.copy2(ORIGINAL_FILE_PATH, COPIED_FILE_PATH)
    
	# Remove the original file
	os.remove(ORIGINAL_FILE_PATH)

	# append ending msg in COPIED_FILE_PATH using shell command
	! echo "<<<END OF TEST>>>" >> {COPIED_FILE_PATH}

# Test class
class TestSimpleAddition(unittest.TestCase):
	# This method will be run before each individual test
	def setUp(self):
		global COUNTER
		COUNTER += 1

	# This method will be run after each individual test
	def tearDown(self):
		# Append the test result to the file
		with open(ORIGINAL_FILE_PATH, 'a') as file:
			result = "PASSED" if self._outcome.success else "FAILED"
			file.write(f"Test {COUNTER}: {result}\n")

	# Test methods
	def test_add_positive_numbers(self):
		self.assertEqual(simple_addition(3, 4), 7)

	def test_add_negative_numbers(self):
		self.assertEqual(simple_addition(-3, -4), -7)

# Running the tests
suite = unittest.TestLoader().loadTestsFromTestCase(TestSimpleAddition)
runner = unittest.TextTestRunner()
runner.run(suite)

# Read the copied file to show the results
with open(COPIED_FILE_PATH, 'r') as result_file:
	test_results = result_file.read()

print(test_results)

# notes:
# 1) '/tmp' is a temporary directory in Unix-based systems (that is always available irrespective of 'cwd')
# 2) to access variables in shell commands, we use '{}' around the variable name (e.g. {COPIED_FILE_PATH})

..


----------------------------------------------------------------------
Ran 2 tests in 0.140s

OK


Test Results:
Test 1: PASSED
Test 2: PASSED
<<<END OF TEST>>>

